Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 917 µs, sys: 3.49 ms, total: 4.4 ms
Wall time: 11.7 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38439 instead
  warnings.warn(


Client Scheduler: inproc://10.1.0.4/6840/1 Dashboard: http://10.1.0.4:38439/status,Cluster Workers: 1 Cores: 6 Memory: 7.29 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 269 ms, sys: 24.5 ms, total: 293 ms
Wall time: 2.49 s


[1.5118137763529522,
 1.9413528084197536,
 1.6455597090146643,
 1.0628854461335429,
 1.7540662723513076,
 1.9439038209038206,
 1.7429529262454238,
 1.4333306970151973,
 1.347701528513303,
 1.4891178241645264,
 1.5244160842233905,
 1.0557422564098187,
 1.4892372362710873,
 1.0069801949806672,
 1.6276520390951854,
 1.1063876981893719,
 1.2322090559705452,
 1.4068466000776452,
 1.072782540186582,
 1.108447441098613,
 1.0327810223683889,
 1.8714484639704683,
 1.4500396601931045,
 1.0860993977798696,
 1.0137010137379165,
 1.8451735536892526,
 1.813707679459029,
 1.6327942353985878,
 1.675605230388753,
 1.342793354414615,
 1.7785834214071525,
 1.7835785460629991,
 1.530983318372848,
 1.3646924906866027,
 1.8559585066847955,
 1.8163782357074694,
 1.561449645427341,
 1.5971583459224892,
 1.7023887289310324,
 1.836616648450983,
 1.7375022096078458,
 1.7450611011519974,
 1.1549229735974293,
 1.6230625787685173,
 1.2418372722369306,
 1.828113221797938,
 1.8092936553094892,
 1.763545013714142,
 1.

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.0733935742903555,
 1.7612664369471291,
 1.9087602691599708,
 1.4902920136414424,
 1.7386454980547845,
 1.8986140681874892,
 1.60823624689902,
 1.3709707351736378,
 1.6799759407783068,
 1.635060237616419,
 1.4775925288065919,
 1.5839193637061706,
 1.3999468828997879,
 1.546905737346962,
 1.068445695371413,
 1.0845606709325843,
 1.7125560192943938,
 1.300637622220024,
 1.0116437744657572,
 1.952292087084635,
 1.4580602238234528,
 1.842965946471589,
 1.3948016170799686,
 1.016524473590119,
 1.056988710528623,
 1.614506323348491,
 1.6545299820133101,
 1.4921892993882655,
 1.4438671982066942,
 1.9740947841844805,
 1.0373919796575215,
 1.9701032378410996,
 1.8952746226153379,
 1.4383874863558552,
 1.578196980981895,
 1.1429527156064294,
 1.0115769311488607,
 1.8746615617663687,
 1.3346188756463777,
 1.0970032802997849,
 1.208611686098894,
 1.2617530085451873,
 1.0585599258279421]

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 224 ms, sys: 18.8 ms, total: 243 ms
Wall time: 2.24 s
